In [191]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import yfinance as yf
import plotly.express as px
import plotly.graph_objects as go

In [192]:
AAPL = yf.download("AAPL", start="2015-1-1", end='2022-1-1')['Adj Close']
MSFT = yf.download("MSFT", start="2015-1-1", end='2022-1-1')['Adj Close']
TSLA = yf.download("TSLA", start="2015-1-1", end='2022-1-1')['Adj Close']
AMZN = yf.download("AMZN", start="2015-1-1", end='2022-1-1')['Adj Close']

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [193]:
stocks = pd.concat([AAPL, MSFT, TSLA, AMZN], axis=1)

In [196]:
def portfolio_optimisation(n, stocks):

  # Logarithmic returns used rather than raw returns due to properties, e.g additivity and symmetry
  log_rets = np.log(stocks/stocks.shift(1))
  log_rets_cov = log_rets.cov()
  
  # Random weight generator
  def weight_generator(n):
    weights = np.random.random(n)
    return weights/np.sum(weights)

  # Annual average return for given weight
  def weighted_rets(log_rets, weights):
    return np.sum(log_rets.mean() * weights) * 252

  # Annual volatility 
  def volatility(log_rets_cov, weights):
    annual_cov = np.dot(log_rets_cov * 252, weights)
    return np.sqrt(np.dot(annual_cov, weights.transpose()))

  # Generate weights and calculate returns, volatility and Sharpe ratio
  mc_port_rets = []
  mc_port_vol = []
  mc_weights = []
  for sim in range(10000):
    weights = weight_generator(n)
    mc_weights.append(weights)
    mc_port_rets.append(weighted_rets(log_rets, weights))
    mc_port_vol.append(volatility(log_rets_cov, weights))

  mc_sharpe_ratios = np.array(mc_port_rets)/np.array(mc_port_vol)

  # Find weightings that maximise Sharpe ratio
  # Note multiplying by -1 and minimising is the same as maximising original
  def minimise_func(weights):
    return -1 * (weighted_rets(log_rets, weights)/volatility(log_rets_cov, weights))
  bounds = tuple((0,1) for i in range(n))
  equal_weights = n * [1/n]
  sum_constraint = ({'type': 'eq', 'fun': lambda weights: np.sum(weights)-1})
  x = minimize(fun=minimise_func,x0=equal_weights,bounds=bounds,constraints=sum_constraint)['x']

  # Calculate optimal volatility for each return
  exp_rets_range = np.linspace(min(mc_port_rets),max(mc_port_rets), 500)
  
  def vol(weights):
    annual_cov = np.dot(log_rets_cov *252, weights)
    vol = np.sqrt(np.dot(weights.transpose(), annual_cov))
    return vol
  
  frontier_vol = []

  for possible_rets in exp_rets_range:

    # Optimisation constraints
    constraints = ({'type':'eq','fun': lambda weights: np.sum(weights)-1},
                   {'type':'eq','fun': lambda weights: weighted_rets(log_rets, weights) - possible_rets})
    result = minimize(vol, equal_weights,bounds=bounds,constraints=constraints)
    frontier_vol.append(result['fun'])

  # Plot efficient frontier
  df = pd.DataFrame({'Volatility': mc_port_vol, 'Return': mc_port_rets, 'Sharpe Ratio': mc_sharpe_ratios})
  fig = px.scatter(df, x = 'Volatility', y = 'Return', color = 'Sharpe Ratio' )
  fig.add_trace(go.Scatter(x = frontier_vol, y = exp_rets_range, name = 'Efficient Frontier'))
  fig.show()

  return print("Weight that optimises Sharpe ratio:", x)

In [197]:
portfolio_optimisation(4, stocks)

Weight that optimises Sharpe ratio: [0.11907115 0.33319709 0.1306099  0.41712186]
